In [61]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [62]:
import os
print(os.getcwd())

C:\Users\guswh\Desktop\data-analysis\dacon_housePrediction\src


In [63]:
train = pd.read_csv('../data/prep/train3_train.csv')
test = pd.read_csv('../data/prep/test3_valid.csv')
submission = pd.read_csv('../data/prep/submission_valid.csv')

In [64]:
train

,id,Overall.Qual,Gr.Liv.Area,Exter.Qual,Garage.Cars,Garage.Area,Kitchen.Qual,Total.Bsmt.SF,X1st.Flr.SF,Bsmt.Qual,...,Year.Built,Year.Remod.Add,Garage.Yr.Blt,Garage.Area.Cars,comb.Qual,Garage.Yr.Blt_cal,Year.Remod.Add_cal,Year.Built_cal,comb.Built,target
0,415,1.609438,6.796824,1.386294,1.098612,5.983936,1.386294,6.796824,6.796824,1.386294,...,7.581210,7.581210,7.595387,5.293305,2.639057,3.178054,3.951244,3.951244,4.836282,11.860062
1,463,2.397895,8.196437,1.609438,1.386294,6.694562,1.609438,7.565793,7.513164,1.791759,...,7.598900,7.599401,7.598900,5.598422,3.178054,2.833213,2.772589,2.833213,3.871201,13.345509
2,179,1.945910,7.581720,1.386294,1.098612,6.329721,1.609438,6.690842,6.690842,1.791759,...,7.601902,7.602401,7.601902,5.638355,2.944439,2.397895,2.302585,2.397895,3.401197,12.278398
3,526,1.791759,6.898715,1.386294,1.098612,6.357842,1.386294,6.898715,6.898715,1.609438,...,7.598900,7.598900,7.599401,5.666427,2.772589,2.772589,2.833213,2.833213,3.871201,11.957618
4,195,1.945910,7.097549,1.609438,1.098612,6.304449,1.386294,7.097549,7.097549,1.609438,...,7.589842,7.589842,7.589842,5.613128,2.890372,3.555348,3.555348,3.555348,4.634729,12.013707
5,938,1.945910,7.177019,1.386294,1.098612,6.210600,1.609438,6.455199,6.472346,1.609438,...,7.599902,7.604396,7.599902,5.519459,2.890372,2.708050,1.791759,2.708050,3.526361,12.072547
6,1142,1.791759,6.872128,1.386294,1.098612,6.665684,1.386294,6.872128,6.872128,1.386294,...,7.581720,7.581720,7.593878,5.973810,2.708050,3.295837,3.931826,3.931826,4.844187,11.831386
7,1323,2.079442,7.449498,1.386294,1.098612,5.579730,1.609438,6.747587,6.766192,1.386294,...,7.564238,7.597396,7.564238,4.890349,2.890372,4.442651,2.995732,4.442651,5.236442,12.380030
8,1253,2.079442,7.180070,1.386294,1.098612,6.206576,1.386294,7.180070,7.180070,1.386294,...,7.580189,7.601902,7.580189,5.515443,2.833213,3.988984,2.397895,3.988984,4.762174,12.133507
9,1268,1.791759,6.959399,1.386294,0.693147,6.505784,1.386294,6.959399,6.959399,1.386294,...,7.584773,7.584773,7.593374,6.505784,2.708050,3.332205,3.806662,3.806662,4.753590,11.639567


In [65]:
train = train.iloc[:, 1:]
test = test.iloc[:, 1:]

In [66]:
X = train.drop('target', axis = 1)
#y = np.log1p(train.target)
y = train.target
target = test[X.columns]

In [67]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from catboost import CatBoostRegressor, Pool
from ngboost import NGBRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold

In [68]:
def NMAE(true, pred) -> float:
    mae = np.mean(np.abs(true - pred))
    score = mae / np.mean(np.abs(true))
    return score

In [69]:
nmae_score = make_scorer(NMAE, greater_is_better=False)

In [70]:
kf = KFold(n_splits = 10, random_state = 42, shuffle = True)

In [54]:
rf_pred = np.zeros(target.shape[0])
rf_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    rf = RandomForestRegressor(random_state = 42, criterion = 'mae')
    rf.fit(tr_x, tr_y)
    
    val_pred = np.expm1(rf.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    rf_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    fold_pred = rf.predict(target) / 10
    rf_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(rf_val)} & std = {np.std(rf_val)}')

1 FOLD Training.....
1 FOLD NMAE = 0.1110464030108552

2 FOLD Training.....
2 FOLD NMAE = 0.10624136085725426

3 FOLD Training.....
3 FOLD NMAE = 0.0930866753117235

4 FOLD Training.....
4 FOLD NMAE = 0.08759134041704819

5 FOLD Training.....
5 FOLD NMAE = 0.12498363571776769

6 FOLD Training.....
6 FOLD NMAE = 0.1015124495558646

7 FOLD Training.....
7 FOLD NMAE = 0.07828731863384829

8 FOLD Training.....
8 FOLD NMAE = 0.09736499114855156

9 FOLD Training.....
9 FOLD NMAE = 0.10781846918693792

10 FOLD Training.....
10 FOLD NMAE = 0.091025010730968

10FOLD Mean of NMAE = 0.09989576545708193 & std = 0.012685306859588267


In [55]:
gbr_pred = np.zeros(target.shape[0])
gbr_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    gbr = GradientBoostingRegressor(random_state = 42, max_depth = 4, learning_rate = 0.05, n_estimators = 1000)
    gbr.fit(tr_x, tr_y)
    
    val_pred = np.expm1(gbr.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    gbr_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    fold_pred = gbr.predict(target) / 10
    gbr_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(gbr_val)} & std = {np.std(gbr_val)}')

1 FOLD Training.....
1 FOLD NMAE = 0.0980893291615858

2 FOLD Training.....
2 FOLD NMAE = 0.09766613872409585

3 FOLD Training.....
3 FOLD NMAE = 0.1026720408341977

4 FOLD Training.....
4 FOLD NMAE = 0.09372756809230672

5 FOLD Training.....
5 FOLD NMAE = 0.11343844911039153

6 FOLD Training.....
6 FOLD NMAE = 0.10156917803809239

7 FOLD Training.....
7 FOLD NMAE = 0.0877587889952045

8 FOLD Training.....
8 FOLD NMAE = 0.0894129958807658

9 FOLD Training.....
9 FOLD NMAE = 0.10502300692937165

10 FOLD Training.....
10 FOLD NMAE = 0.08896727431232107

10FOLD Mean of NMAE = 0.0978324770078333 & std = 0.0077494321634103995


In [56]:
cb_pred = np.zeros(target.shape[0])
cb_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    tr_data = Pool(data = tr_x, label = tr_y)
    val_data = Pool(data = val_x, label = val_y)
    
    cb = CatBoostRegressor(depth = 4, random_state = 42, loss_function = 'MAE', n_estimators = 3000, learning_rate = 0.03, verbose = 0)
    cb.fit(tr_data, eval_set = val_data, early_stopping_rounds = 750, verbose = 1000)
    
    val_pred = np.expm1(cb.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    cb_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    target_data = Pool(data = target, label = None)
    fold_pred = cb.predict(target) / 10
    cb_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(cb_val)} & std = {np.std(cb_val)}')

1 FOLD Training.....
0:	learn: 0.2924583	test: 172452.1526702	best: 172452.1526702 (0)	total: 12.8ms	remaining: 38.4s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 172452.1527
bestIteration = 0

Shrink model to first 1 iterations.
1 FOLD NMAE = 0.2549068375163498

2 FOLD Training.....
0:	learn: 0.2888077	test: 195683.6488472	best: 195683.6488472 (0)	total: 1.05ms	remaining: 3.15s
1000:	learn: 0.0606667	test: 195683.5505495	best: 195683.5491124 (467)	total: 1.19s	remaining: 2.38s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 195683.5491
bestIteration = 467

Shrink model to first 468 iterations.
2 FOLD NMAE = 0.09084969777282487

3 FOLD Training.....
0:	learn: 0.2853030	test: 198387.0535317	best: 198387.0535317 (0)	total: 1.3ms	remaining: 3.9s
1000:	learn: 0.0599259	test: 198386.9228408	best: 198386.9217100 (329)	total: 1.41s	remaining: 2.82s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 198386.9217
bestIteration = 329

Shrin

In [71]:
# ngboost
ngb_pred = np.zeros(target.shape[0])
ngb_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    print(f'{n + 1} FOLD Training.....')
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    ngb = NGBRegressor(random_state = 42, n_estimators = 1000, verbose = 0, learning_rate = 0.03)
    ngb.fit(tr_x, tr_y, val_x, val_y, early_stopping_rounds = 300)
    
    val_pred = np.expm1(ngb.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    ngb_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    target_data = Pool(data = target, label = None)
    fold_pred = ngb.predict(target) / 10
    ngb_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(ngb_val)} & std = {np.std(ngb_val)}')

1 FOLD Training.....
1 FOLD NMAE = 0.09906484672392254

2 FOLD Training.....
2 FOLD NMAE = 0.09972696492359634

3 FOLD Training.....
3 FOLD NMAE = 0.10212299539672215

4 FOLD Training.....
4 FOLD NMAE = 0.09526636651037737

5 FOLD Training.....
5 FOLD NMAE = 0.0872656139962474

6 FOLD Training.....
6 FOLD NMAE = 0.10225604687261712

7 FOLD Training.....
7 FOLD NMAE = 0.10018640002306975

8 FOLD Training.....
8 FOLD NMAE = 0.10681859387843677

9 FOLD Training.....
9 FOLD NMAE = 0.09013587280087337

10 FOLD Training.....
10 FOLD NMAE = 0.10178997381990769

10FOLD Mean of NMAE = 0.09846336749457704 & std = 0.00564392039465687


In [74]:
submission['target'] = ngb_pred

In [75]:
submission

,id,target
0,1,13.203626
1,4,11.689749
2,7,11.977865
3,12,12.164013
4,14,11.550175
5,15,12.437183
6,21,11.753482
7,22,11.839081
8,27,11.598581
9,33,11.897236


In [76]:
submission.to_csv("../out/ngb/ngb1_valid.csv", header = True, index = False)